In [1]:
%matplotlib inline 

import matplotlib.pyplot as plt 
import pandas 
from sklearn import preprocessing, svm, metrics, pipeline, cross_validation, decomposition, grid_search

## SVM for expression using enzyme design features 

In [2]:
# # features 

# ! mkdir feature_sets

# feature_set = [ str( i ) for i in [ 
#     u'total_score', u'fa_rep', u'hbond_sc', u'tot_pstat_pm', u'tot_nlpstat_pm', u'tot_burunsat_pm', u'tot_hbond_pm', u'tot_NLconts_pm', u'tot_nlsurfaceE_pm', 
#     u'SR_1_total_score', u'SR_1_fa_rep', u'SR_1_hbond_sc', u'SR_1_hbond_pm', u'SR_1_burunsat_pm', u'SR_1_pstat_pm', u'SR_1_nlpstat_pm', 
#     u'SR_2_total_score', u'SR_2_fa_rep', u'SR_2_hbond_sc', u'SR_2_hbond_pm', u'SR_2_burunsat_pm', u'SR_2_pstat_pm', u'SR_2_nlpstat_pm', 
#     u'SR_3_total_score', u'SR_3_fa_rep', u'SR_3_hbond_sc', u'SR_3_hbond_pm', u'SR_3_burunsat_pm', u'SR_3_pstat_pm', u'SR_3_nlpstat_pm', 
#     u'SR_4_total_score', u'SR_4_fa_rep', u'SR_4_hbond_sc', u'SR_4_hbond_pm', u'SR_4_burunsat_pm', u'SR_4_pstat_pm', u'SR_4_nlpstat_pm', 
#     u'SR_5_total_score', u'SR_5_fa_rep', u'SR_5_hbond_sc', u'SR_5_interf_E_1_2', u'SR_5_dsasa_1_2', u'SR_5_hbond_pm', u'SR_5_burunsat_pm'   
# ] ]


# for eature in feature_set:
#     f = pandas.read_csv( '/Users/alex/Documents/bagel-benchmark/data_sets/calculated/enzyme_design_talaris_2014.csv', usecols=feature_set+['name'] )
#     f.groupby( 'name' ).apply( lambda x: x.sort_values( eature ).head( 10 ).mean() ).to_csv( 'feature_sets/{}.csv'.format( eature ) )

In [29]:
# experimental values 

E = pandas.read_csv( '../data_set.csv', index_col='mutant', usecols=['mutant','expression'] ) 

# features 

F = pandas.read_csv( '/Users/alex/Documents/bagel-benchmark/data_sets/calculated/enzyme_design_talaris_2014.csv', index_col=0 )
F = F.groupby( 'name' ).apply( lambda x: x.head(5) ) 
F.index = F.name 
J = F.join( E ).dropna().copy()

In [22]:
# machine learning setup 

X = J.drop( [ 'name', 'description', 'expression' ], axis=1 ) 
y = J[ 'expression' ] 

print X.shape
print y.shape

(535, 59)
(535,)


In [24]:
pln = pipeline.Pipeline([
    ( 'scaler', preprocessing.StandardScaler() ), 
    ( 'pca', decomposition.PCA() ), 
    ( 'svm', svm.SVC() ), 
])

param_grid = {
    'svm__kernel': [ 'rbf' ], 
    #'svm__C': [ 0.003, 0.03, 0.3, 3, 30, 300, 3000 ],
    #'svm__gamma': [ 0.003, 0.03, 0.3, 3, 30, 300, 3000 ],
    'svm__class_weight': [ 'balanced', None ], 
    #'pca__n_components': [ 2, 4, 8 ], 
    #'pca__whiten': [ True, False ], 
}

skf = cross_validation.StratifiedKFold( y, n_folds=6 )
gs = grid_search.GridSearchCV( pln, param_grid, cv=skf )
gs.fit( X, y ) 

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[ 1.  1. ...,  1.  1.], n_folds=6, shuffle=False, random_state=None),
       error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'svm__class_weight': ['balanced', None], 'svm__kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [25]:
print gs.best_estimator_
print gs.best_estimator_.predict( X ) 
print gs.best_estimator_.score( X, y ) 

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1. 

In [26]:
# now for the blind data 

E = pandas.read_csv( '../blind_data.csv', index_col='name' ) 
F = pandas.read_csv( 'feature_sets/total_score.csv', index_col='name' )

blind = F.join( E ).dropna()

X_test = blind.drop( 'expression', axis=1 )
y_test = blind.expression

In [27]:
gs.predict( X_test ) 

ValueError: operands could not be broadcast together with shapes (22,44) (59,) (22,44) 

In [28]:
tpr, fpr, thresh = metrics.roc_curve( y_test, gs.decision_function( X_test ) )

plt.figure( figsize=( 4,4 ) ) 
plt.plot( tpr, fpr, label='blind test set', c='orange' ) 
plt.legend(loc='lower right')
plt.xlabel( 'False positive rate' )
plt.ylabel( 'True positive rate' ) 

print 'score={}'.format( gs.score( X_test, y_test ) ) 

ValueError: operands could not be broadcast together with shapes (22,44) (59,) (22,44) 

-----------

## Predicting Tm 

In [25]:
df = pandas.read_csv( '../data_set.csv', index_col='mutant' ) 
features = pandas.read_csv( '/Users/alex/Documents/bagel-benchmark/data_sets/calculated/enzyme_design_talaris_2014.csv', index_col='name' )

J = df.join( features )
J[ 'mutant_name' ] = J.index 
G = J.groupby( 'mutant_name' ).apply( lambda x: x.sort_values( 'total_score' ).head( 10 ).mean() ) 

feature_set = [
    u'tm',
    u'total_score', u'fa_rep', u'hbond_sc', u'tot_pstat_pm', u'tot_nlpstat_pm', u'tot_burunsat_pm', u'tot_hbond_pm', u'tot_NLconts_pm', u'tot_nlsurfaceE_pm', 
    u'SR_1_total_score', u'SR_1_fa_rep', u'SR_1_hbond_sc', u'SR_1_hbond_pm', u'SR_1_burunsat_pm', u'SR_1_pstat_pm', u'SR_1_nlpstat_pm', 
    u'SR_2_total_score', u'SR_2_fa_rep', u'SR_2_hbond_sc', u'SR_2_hbond_pm', u'SR_2_burunsat_pm', u'SR_2_pstat_pm', u'SR_2_nlpstat_pm', 
    u'SR_3_total_score', u'SR_3_fa_rep', u'SR_3_hbond_sc', u'SR_3_hbond_pm', u'SR_3_burunsat_pm', u'SR_3_pstat_pm', u'SR_3_nlpstat_pm', 
    u'SR_4_total_score', u'SR_4_fa_rep', u'SR_4_hbond_sc', u'SR_4_hbond_pm', u'SR_4_burunsat_pm', u'SR_4_pstat_pm', u'SR_4_nlpstat_pm', 
    u'SR_5_total_score', u'SR_5_fa_rep', u'SR_5_hbond_sc', u'SR_5_interf_E_1_2', u'SR_5_dsasa_1_2', u'SR_5_hbond_pm', u'SR_5_burunsat_pm'   
]